In [6]:
import pandas as pd
import numpy as np
import time
import re as re
from selenium import webdriver
import translators as ts
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

    
driver = webdriver.Chrome()
def get_data_nordea(url):
    
    """
    This function will collect the grant information provided from the Nordea foundation grant database with the url: https://nordeafonden.dk/det-har-vi-stoettet/alle-uddelinger
    
    The paths are found using devtools, and found through the webpage.
    
    note: When the window opens, one must actively choose a web browser (eg. Ecosia) and close the
    the Cookie popup window - to start the extraction. 
    
    The information is saved in the lists of the following groups:
    project_category_l = []
    desc_l = []
    rec_l = []
    year_l = []
    region_l = []
    amount_l = []
    sub_category_l = []
    
    and if data is not avaiable for a certain group, then "NA" is inserted. 
    
    The lists are finally zipped together in the biglist dataframe.
    
    input: target_url (https://nordeafonden.dk/det-har-vi-stoettet/alle-uddelinger)
    return: Dataframe with text extracted
    """
    
    project_category_l = []
    desc_l = []
    rec_l = []
    year_l = []
    region_l = []
    amount_l = []
    sub_category_l = []
    
    
    
    driver.get(url)
    driver.maximize_window() #opening full web browser window
    time.sleep(5)
    
    #hradcoded the first grant because it's structure is different from the rest
    
    ActionChains(driver).move_to_element(driver.find_element(By.XPATH, f"/html/body/div[1]/main/div/div/div/div/div[3]/div/div[5]/table/tr[1]/td/div/div/div[1]/div")).perform()
    time.sleep(2)
    
    project_category = driver.find_element(By.XPATH, "/html/body/div[1]/main/div/div/div/div/div[3]/div/div[5]/table/tr[3]/td/div/div/div[1]/div").text
    description = driver.find_element(By.XPATH, "/html/body/div[1]/main/div/div/div/div/div[3]/div/div[5]/table/tr[1]/td/div/div/div[2]/h3").text
    receiver = driver.find_element(By.XPATH, "/html/body/div[1]/main/div/div/div/div/div[3]/div/div[5]/table/tr[1]/td/div/div/div[3]").text
    year = driver.find_element(By.XPATH, "/html/body/div[1]/main/div/div/div/div/div[3]/div/div[5]/table/tr[1]/td/div/div/div[4]/div[1]/div[1]/time").text
    region = driver.find_element(By.XPATH, "/html/body/div[1]/main/div/div/div/div/div[3]/div/div[5]/table/tr[1]/td/div/div/div[4]/div[1]/div[2]").text
    amount = driver.find_element(By.XPATH, "/html/body/div[1]/main/div/div/div/div/div[3]/div/div[5]/table/tr[1]/td/div/div/div[4]/div[2]").text
    sub_category = driver.find_element(By.XPATH, "/html/body/div[1]/main/div/div/div/div/div[3]/div/div[5]/table/tr[1]/td/div/div/div[4]/div[1]/div[3]").text

    project_category_l.append(project_category)
    desc_l.append(description)
    rec_l.append(receiver)
    region_l.append(region)
    amount_l.append(amount)
    year_l.append(year)
    sub_category_l.append(sub_category)
    
    
    for i in range(3,3325): #3325

        try:
            

            ActionChains(driver).move_to_element(driver.find_element(By.CSS_SELECTOR, f"#ot-maps-result-table > tr:nth-child({i}) > td > div > div > div.group-bottom-wrapper")).perform()  #move to place
            time.sleep(2)
            
            #project category
                
            test_function(project_category_l, f"#ot-maps-result-table > tr:nth-child({i}) > td > div > div > div.slugline > div")
    
            #description
        
            test_function(desc_l, f"#ot-maps-result-table > tr:nth-child({i}) > td > div > div > div.field.node-title")

            #receiver
            
            test_function(rec_l, f"#ot-maps-result-table > tr:nth-child({i}) > td > div > div > div.field.field-organisation")

            #year
            
            test_function(year_l, f"#ot-maps-result-table > tr:nth-child({i}) > td > div > div > div.group-bottom-wrapper > div.group-info-wrapper > div.field.field-date-notime")

            #region
            
            test_function(region_l, f"#ot-maps-result-table > tr:nth-child({i}) > td > div > div > div.group-bottom-wrapper > div.group-info-wrapper > div.field.field-region")

            #amount
            
            test_function(amount_l, f"#ot-maps-result-table > tr:nth-child({i}) > td > div > div > div.group-bottom-wrapper > div.group-amount-wrapper > div")

            #sub_category
            
            test_function(sub_category_l, f"#ot-maps-result-table > tr:nth-child({i}) > td > div > div > div.group-bottom-wrapper > div.group-info-wrapper > div.field.field-category")
            

            #ActionChains(driver).move_to_element(driver.find_element(By.XPATH, "/html/body/main/section[2]/div/div/div[7]/div/div[2]/a[",i+1,"]/div[4]")).perform()  #move to place
            
            if i == 32:
                ActionChains(driver).move_to_element(driver.find_element(By.CSS_SELECTOR, "#ot-maps-result-table-fetch-more")).perform()  #move to place
                element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"#ot-maps-result-table-fetch-more")))
                element.click()
                print("i number:", i)
                time.sleep(7)
                
            elif i > 35 and i % 30 == 2:
                ActionChains(driver).move_to_element(driver.find_element(By.CSS_SELECTOR, "#ot-maps-result-table-fetch-more")).perform()  #move to place
                element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"#ot-maps-result-table-fetch-more")))
                element.click()
                print("i number:", i)
                time.sleep(7)
            
        except NoSuchElementException: #avoid programme crashing
            print("Loop ended")
            print(i)



    df_nordea = pd.DataFrame(list(zip(year_l, rec_l, amount_l, desc_l, region_l, project_category_l, sub_category_l)), columns=["Year",'Reciever','Grant size (DKK)', 'Description', "Region", "Project Category", "Project Subcategory"])

    return df_nordea

def test_function(listname, css_selector_p):
    
    """
    helper function, that will first try to find the right element, 
    and if an error is raied, it will insert an "NA" instead
    
    """
    try:
        text_found = driver.find_element(By.CSS_SELECTOR, css_selector_p).text
        listname.append(text_found)
    except ElementClickInterceptedException:
        text_found = "NA"
        listname.append(location)
    except StaleElementReferenceException:
        text_found = "NA"
        listname.append(location)
    except NoSuchElementException:
        text_found = "NA"
        listname.append(text_found)
        
        return

biglist = get_data_nordea("https://nordeafonden.dk/det-har-vi-stoettet/alle-uddelinger")
pd.set_option('display.max_columns', None)

biglist.to_csv("csv_nordea_tab.csv", sep = "\t")